#### 导入科学包

In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
# 加载数据集
pre = 'F:/University/学期资料/大二下资料/泰迪杯/dataset/5scan/'
venue = pd.read_csv('F:/University/学期资料/大二下资料/泰迪杯/dataset/3venue.csv')
acid = pd.read_csv('F:/University/学期资料/大二下资料/泰迪杯/dataset/6acid.csv')


In [ ]:
### 定义函数： 给定key-->list，建立一个所有键值为 0 的字典
def todict(x):
    my_set = x
    my_dict = {key: 0 for key in my_set}
    return my_dict
# 筛选出阳性者的核酸检测记录
acid_pos = acid.drop(acid[acid['jg'] == '阴性'].index)
acid_pos = acid_pos.drop(acid[acid['jg'] == '未知'].index)
acid_pos

#### 建立变量
acid_pos: 所有核酸检测结果为阳性的人员数据，包含人员编号，采样时间和采样地点

all_venues: 所有的场地信息

all_positive_id: 所有阳性人员的编号

all_grids: 所有的场地编号，（仅用）便于建立下面两个字典变量

flow_of_each: 字典类型，key为地点编号，value为某一地点共61天的人流量

all_po_grids_dict: 字典，key为地点编号，value为某一地点共61天的阳性人员流动量

venue_list: 将所有地点编号转换为list类型，用于做上两个字典的索引而设

mod_ob_flow: 模型需使用的目标feature - 1， 因为场所编号从1开始，便于索引用None初始化第0个元素，对应使用变量flow_of_each的数据，使用举例：获取某一个场所id的61天平均人流 -->
mod_ob_flow[场所ID]

mod_ob_pos: 模型需使用的目标feature - 2， 同上

In [ ]:
# 筛选出阳性者的核酸检测记录
acid_pos = acid.drop(acid[acid['jg'] == '阴性'].index)
acid_pos = acid_pos.drop(acid[acid['jg'] == '未知'].index)
acid_pos = acid_pos[['user_id', 'cysj', 'grid_point_id']]

all_venues = venue
all_positive_id = acid.user_id.unique()
files = os.listdir(pre)

all_grids = set(all_venues['grid_point_id'].tolist())
all_po_grids_dict = todict(all_grids)
flow_of_each = todict(all_grids)

keys = flow_of_each.keys()
venue_list = list(keys)

mod_ob_flow = [None]
mod_ob_pos = [None]

##### 访问某一个场所的61天数据： mod_ob[venue_list] = list_array {....}
##### 开始查找

In [ ]:
idx = 0
## 遍历每一天的数据集
for file_name in files:
    df_scan = pd.read_csv(pre + file_name)
    e = df_scan[['user_id', 'grid_point_id', 'create_time']]
    sample_slice_id = []  ## 每一天人在两天后的核酸检测结果为阳性的人员id
    ## 遍历第一次获取目标时间范围点
    for _, row in e.iterrows():
        enter_time = row[2][:10]  ## 扫码时间 13
        obj_time = pd.to_datetime(enter_time, format='%Y-%m-%d')  ## 扫码时间后两天-- 核酸检测当天 15
        obj_time_m = obj_time + pd.Timedelta(days=1)  ## 核酸检测时间结束16
        mask = (pd.to_datetime(acid_pos['cysj']) >= obj_time) & (pd.to_datetime(acid_pos['cysj']) <= obj_time_m)
        sample_slice_id = acid_pos[mask]['user_id'].to_list()  ## 获得此时 将在两天后为阳性 的人员id
        break

    ## 每一天中每一行
    for _, row in e.iterrows():
        grid_id = row[1]  ## 扫码地点
        flow_of_each[grid_id] += 1  ## 人流加一

        if row[0] in sample_slice_id:  ## 该人若在目标list中 (阳性)
            place_id = grid_id
            all_po_grids_dict[place_id] += 1

    print(idx)
    idx += 1

In [ ]:
for id in all_grids:
    print('场所编号', id, "{ 人流量: ", flow_of_each[id], ",\t阳性人员流动量: ", all_po_grids_dict[id], "}")

#### 第一阶段：建立特征值
###### 导入模型

In [ ]:
mod_ob_flow = flow_of_each
mod_ob_pos = all_po_grids_dict
# 特征1：特定场所出现阳性人员数量
feature_1 = pd.DataFrame([mod_ob_pos])
# 特征2：场所人数
feature_2 = pd.DataFrame([mod_ob_flow])

feature_1 = feature_1.transpose()
feature_2 = feature_2.transpose()
# 检查feature_2中哪些行的数值为0
zero_mask = (feature_2 == 0)

In [3]:
# 读取数据
data = pd.read_table("cordinates.csv")

In [ ]:
print(data)

In [4]:
# 计算欧几里得距离矩阵
def euclidean_distance_matrix(data):
    n = len(data)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distances[i, j] = np.sqrt((data.iloc[i, 1] - data.iloc[j, 1])**2
                                      + (data.iloc[i, 2] - data.iloc[j, 2])**2)
    return distances

In [ ]:
# 计算空间权重矩阵
def spatial_weight_matrix(data):
    distances = euclidean_distance_matrix(data)
    weights = np.zeros((len(data), len(data)))
    for i in range(len(data)):
        for j in range(len(data)):
            if i == j:
                weights[i, j] = 0
            else:
                weights[i, j] = 1 / distances[i, j] if distances[i, j] != 0 else 0
    return weights

In [ ]:
# 计算Moran's I指标
def morans_I(data):
    x = data.iloc[:, 3]
    n = len(x)
    w = spatial_weight_matrix(data)
    wx = w @ x
    sum_x = np.sum(x)
    sum_w = np.sum(w)
    sum_wx = np.sum(wx)
    mean_x = sum_x / n
    s0 = np.sum((x - mean_x)**2)
    s1 = np.sum(wx**2) / sum_w
    s2 = (sum_wx / sum_w)**2
    I = (n / s0) * ((s1 - s2) / (s0 / (n-1)))
    return I, w

#### 输出结果

In [ ]:
I, w = morans_I(data)
print(I)
print(w)

In [ ]:
def count_neighbors(data, distance_threshold):
    n = len(data)
    neighbor_counts = np.zeros(n)
    neighbor_positions = [[] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            distance = np.sqrt((data.iloc[i, 1] - data.iloc[j, 1])**2
                                  + (data.iloc[i, 2] - data.iloc[j, 2])**2)
            if distance <= distance_threshold:
                neighbor_counts[i] += 1
                neighbor_positions[i].append(j)
    return neighbor_counts, neighbor_positions

In [ ]:
def spatial_weight_matrix(data):
    distances = euclidean_distance_matrix(data)
    weights = np.zeros((len(data), len(data)))
    for i in range(len(data)):
        for j in range(len(data)):
            if i == j:
                weights[i, j] = 0
            else:
                if distances[i, j] == 0:
                    weights[i, j] = 0
                else:
                    weights[i, j] = 1 / distances[i, j]
    return weights

In [ ]:
def local_morans_I(data, neighbor_counts, neighbor_positions):
    x = data.iloc[:, 3]
    n = len(x)
    w = spatial_weight_matrix(data)
    wx = w @ x
    sum_x = np.sum(x)
    sum_w = np.sum(w)
    sum_wx = np.sum(wx)
    mean_x = sum_x / n
    epsilon = 1e-5 # 增加epsilon的值
    s0 = np.sum((x - mean_x)**2) + epsilon
    s1 = np.sum(wx**2) / sum_w
    s2 = (sum_wx / sum_w)**2
    I = (n / s0) * ((s1 - s2) / (s0 / (n-1)))
    local_I = np.zeros(n)
    for i in range(n):
        if neighbor_counts[i] == 0:
            continue
        xi_bar = np.mean(x.iloc[neighbor_positions[i]]) # 修改这一行，使用iloc
        s0i = np.sum((x.iloc[neighbor_positions[i]] - xi_bar)**2) # 修改这一行，使用iloc
        s1i = np.sum(w[i, neighbor_positions[i]] * wx[neighbor_positions[i]])
        s2i = (np.sum(w[i, neighbor_positions[i]]) * xi_bar)**2
        local_I[i] = (neighbor_counts[i] / s0i) * ((s1i - s2i) / (s0i / (neighbor_counts[i]-1)))
    return local_I

In [ ]:
radius = 800
neighbor_counts, neighbor_positions = count_neighbors(data, radius)
local_I = local_morans_I(data, neighbor_counts, neighbor_positions)
print(local_I)